In [ ]:
%matplotlib inline
import random
import torch

In [ ]:
def synthetic_data(w, b, num_examples):
    X = torch.normal(0, 1, (num_examples, len(w)))
    y = torch.matmul(X, w) + b
    y += torch.normal(0, 0.01, y.shape)
    return X, y.reshape((-1, 1))

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
print('features:', features[0], '\nlabel:', labels[0])

In [ ]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))
    # 将list打乱，使样本是随机读取的，没有特定的顺序
    random.shuffle(indices)
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(indices[i:min(i + batch_size, num_examples)])
        yield features[batch_indices],labels[batch_indices]

In [ ]:
batch_size = 10
# for X, y in data_iter(batch_size, features, labels):
#     print(X, '\n', y)
# break

In [ ]:
w = torch.normal(0, 0.01, size=(2, 1),requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [ ]:
def linreg(X, w, b):
    # 线性回归模型
    return torch.matmul(X, w) + b

In [ ]:
def squared_loss(y_hat, y):
    return (y_hat - y.reshape(y_hat.shape))**2 / 2

In [ ]:
def sgd(params, lr, batch_size):
    # 不需要计算梯度，也不会反向传播
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

In [ ]:
lr = 0.01
num_epochs = 5
net = linreg
loss = squared_loss

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):
        l = loss(net(X, w, b), y) # `X`和`y`的⼩批量损失
        # 因为`l`形状是(`batch_size`, 1)，⽽不是⼀个标量。`l`中的所有元素被加到⼀起，
        # 并以此计算关于[`w`, `b`]的梯度
        l.sum().backward()
        sgd([w, b], lr, batch_size) # 使⽤参数的梯度更新参数
        with torch.no_grad():
            train_l = loss(net(features, w, b), labels)
            print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

In [ ]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')


# 简洁实现

In [15]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [16]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

In [17]:
def load_array(data_arrays, batch_size, is_train=True): #@save
    """构造⼀个PyTorch数据迭代器。"""
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

In [18]:
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))


[tensor([[ 0.1475, -0.3858],
         [ 1.1048, -0.7418],
         [ 0.3177,  0.8054],
         [ 0.4911,  0.2038],
         [ 0.5325,  1.0691],
         [ 1.0366,  0.7083],
         [ 0.1165, -0.4508],
         [-0.2199, -0.3959],
         [-0.2366, -0.2256],
         [ 0.0982, -0.6619]]),
 tensor([[5.8137],
         [8.9290],
         [2.1067],
         [4.5116],
         [1.6424],
         [3.8687],
         [5.9688],
         [5.0981],
         [4.4935],
         [6.6534]])]

In [19]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

In [20]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

In [21]:
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

In [22]:
num_epochs = 3
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features), labels)
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000345
epoch 2, loss 0.000104
epoch 3, loss 0.000104
